In [12]:
import json
import faiss 
import torch
import numpy as np
from tqdm import trange
from sentence_transformers import SentenceTransformer

In [1]:
print("11")

11


### Load the Embedding Model
### 加载embedding模型

In [25]:
# embedding_model = "./model/nomic-embed-text-v1"
embedding_model = SentenceTransformer('nomic-ai/nomic-embed-text-v1', trust_remote_code=True)
# embedding_model = SentenceTransformer(embedding_model, trust_remote_code=True)

embedding_model.to(torch.device('cuda'))

!!!!!!!!!!!!megablocks not available, using torch.matmul instead
<All keys matched successfully>


SentenceTransformer(
  (0): Transformer({'max_seq_length': 8192, 'do_lower_case': False}) with Transformer model: NomicBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

### Read the pre-prepared JSON file in the format required by TinyDB (refer to arxiv_paper_db.json for the specific format). Each index corresponds to a single paper.
### 读取准备好符合tinydb要求json文件（具体格式参考arxiv_paper_db.json）, 每个索引index对应一篇paper

In [16]:
with open('E:/AutoGenSurvey/AutoSurvey-main/database1/arxiv_paper_db.json','r') as f:
    papers = json.loads(f.read())
papers_l = list(papers['cs_paper_info'].items())

In [17]:
papers_l[:10]

[('1',
  {'id': '1905.13319',
   'title': 'MathQA: Towards Interpretable Math Word Problem Solving with Operation-Based Formalisms',
   'url': 'https://arxiv.org/pdf/1905.13319.pdf',
   'date': '2019-05-30',
   'abs': 'We introduce a large-scale dataset of math word problems and an interpretable neural math problem solver that learns to map problems to operation programs. Due to annotation challenges, current datasets in this domain have been either relatively small in scale or did not offer precise operational annotations over diverse problem types.',
   'cat': 'cs.CL',
   'authors': ['Aida Amini',
    'Saadia Gabriel',
    'Peter Lin',
    'Rik Koncel-Kedziorski',
    'Yejin Choi',
    'Hannaneh Hajishirzi']}),
 ('2',
  {'id': '2009.11506',
   'title': 'Ape210K: A Large-Scale and Template-Rich Dataset of Math Word Problems',
   'url': 'https://arxiv.org/pdf/2009.11506.pdf',
   'date': '2020-09-24',
   'abs': 'We release a new large-scale and template-rich math word problem dataset na

## Get embeddings of abs and title
## 对title和abs做embedding

In [18]:
def get_embeddings(text_l, batch_size=32):
    res = []
    for i in trange(0, len(text_l), batch_size):
        batch_text = ['search_document: ' + _ for _ in text_l[i:i+batch_size]]
        res.append(embedding_model.encode(batch_text))
    return np.concatenate(res,axis=0)

In [19]:
title_l = [paper[1]['title'] for paper in papers_l]
abs_l = [paper[1]['abs'] for paper in papers_l]

In [26]:
title_embeddings = get_embeddings(title_l)

100%|██████████| 1/1 [00:01<00:00,  1.15s/it]


In [27]:
abs_embeddings = get_embeddings(abs_l)

100%|██████████| 1/1 [00:00<00:00,  6.17it/s]


### Convert embeddings into faiss-index
### 将向量储存为faiss index

In [28]:
title_index = faiss.IndexFlatL2(title_embeddings.shape[1])
title_index.add(title_embeddings)

abs_index = faiss.IndexFlatL2(abs_embeddings.shape[1])
abs_index.add(abs_embeddings)

### Save faiss-index, replacing the .bin file in the database folder.
### 向量保存到本地，替换掉database文件夹中的.bin文件

In [29]:
faiss.write_index(faiss.index_gpu_to_cpu(title_index), 'titles.index')

faiss.write_index(faiss.index_gpu_to_cpu(abs_index), 'abstracts.index')

### Save the mapping from paper ID to index locally, replacing the arxivid_to_index_abs.json file.
### 将paper id到索引的映射保存到本地，替换掉 arxivid_to_index_abs.json文件

In [30]:
paperid_2_index = {}
for paper in papers_l:
    paper_id = paper[1]['id']
    index = paper[0]
    paperid_2_index[paper_id] = int(index)
with open('./paperid_to_index.json', 'w') as f:
    json.dump(paperid_2_index, f, indent=4)

### Modify the file paths in the __init__ fuction within src/database.py.
### 对src/database.py中的__init__部分的初始化文件路径做相应的修改